In [8]:
import yfinance as yf
import pandas as pd
import ta

from pycaret.regression import *


oro = yf.Ticker("GLD")

ModuleNotFoundError: No module named 'distutils'

In [ ]:
!pip install distutils

ERROR: Could not find a version that satisfies the requirement distutils (from versions: none)
ERROR: No matching distribution found for distutils

[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
oro_data = oro.history(period="1y", interval="1d")
oro_data


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2024-10-07 00:00:00-04:00,244.580002,244.820007,243.809998,244.169998,3849000,0.0,0.0,0.0
2024-10-08 00:00:00-04:00,243.789993,244.039993,240.630005,242.369995,7669800,0.0,0.0,0.0
2024-10-09 00:00:00-04:00,241.160004,241.839996,240.639999,241.050003,4120500,0.0,0.0,0.0
2024-10-10 00:00:00-04:00,241.770004,243.020004,241.509995,242.820007,4792700,0.0,0.0,0.0
2024-10-11 00:00:00-04:00,244.500000,245.919998,244.470001,245.470001,5789500,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2025-09-30 00:00:00-04:00,351.130005,355.570007,350.869995,355.470001,13312400,0.0,0.0,0.0
2025-10-01 00:00:00-04:00,356.170013,356.970001,354.790009,356.029999,15573600,0.0,0.0,0.0
2025-10-02 00:00:00-04:00,358.260010,358.670013,351.399994,354.790009,16162100,0.0,0.0,0.0


In [ ]:
oro_data_filtrado = oro_data["Close"]

In [ ]:
df = pd.DataFrame(oro_data_filtrado)

In [ ]:
df['SMA_20'] = ta.trend.sma_indicator(df['Close'], window=20)

# Calcular EMA (media móvil exponencial de 20 días)
df['EMA_20'] = ta.trend.ema_indicator(df['Close'], window=20)

# Calcular RSI (Relative Strength Index de 14 días)
df['RSI_14'] = ta.momentum.rsi(df['Close'], window=14)


In [22]:
df = df.dropna()

In [23]:
df

,Close,SMA_20,EMA_20,RSI_14
Date,,,,
2024-11-01 00:00:00-04:00,252.470001,249.448000,250.676757,58.486820
2024-11-04 00:00:00-05:00,252.830002,249.881001,250.881828,59.349644
2024-11-05 00:00:00-05:00,253.399994,250.432500,251.121653,60.740962
2024-11-06 00:00:00-05:00,245.699997,250.665000,250.605305,40.549930
2024-11-07 00:00:00-05:00,249.649994,251.006499,250.514323,49.773529
...,...,...,...,...
2025-09-30 00:00:00-04:00,355.470001,339.236000,338.945415,80.159237
2025-10-01 00:00:00-04:00,356.029999,340.630499,340.572518,80.483441
2025-10-02 00:00:00-04:00,354.790009,342.035500,341.926565,77.465002


In [24]:
retornos = df.pct_change()

In [26]:
retornos = retornos.dropna()

In [27]:
retornos

,Close,SMA_20,EMA_20,RSI_14
Date,,,,
2024-11-04 00:00:00-05:00,0.001426,0.001736,0.000818,0.014752
2024-11-05 00:00:00-05:00,0.002254,0.002207,0.000956,0.023443
2024-11-06 00:00:00-05:00,-0.030387,0.000928,-0.002056,-0.332412
2024-11-07 00:00:00-05:00,0.016077,0.001362,-0.000363,0.227463
2024-11-08 00:00:00-05:00,-0.006769,0.000496,-0.000971,-0.066716
...,...,...,...,...
2025-09-30 00:00:00-04:00,0.008540,0.004423,0.005158,0.022889
2025-10-01 00:00:00-04:00,0.001575,0.004111,0.004800,0.004045
2025-10-02 00:00:00-04:00,-0.003483,0.004125,0.003976,-0.037504
